## 🖥️ [로컬 실행용] KcELECTRA 악성댓글 분류 모델 학습

> ⚠️ **이 노트북은 로컬 환경(Windows + GPU)용으로 수정되었습니다.**  
> Google Colab에서 실행하지 마세요!

### 실행 전 필수 사항
1. **Anaconda 환경 활성화**: `conda activate kcelectra`
2. **데이터 파일 확인**: `C:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\최종 데이터셋` 폴더에 CSV 파일이 있어야 합니다.
3. **첫 번째 셀(패키지 설치)을 반드시 먼저 실행**하세요.

---

### 1️⃣ 패키지 설치 (최초 1회만 실행)

In [1]:
# ========================================
# 📦 패키지 설치 (최초 1회만 실행)
# ========================================
# 이미 설치했다면 이 셀은 건너뛰어도 됩니다.

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers==4.42.0 datasets accelerate scikit-learn pandas
%pip install -U datasets

print("✅ 패키지 설치 완료!")

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ 패키지 설치 완료!


In [2]:
# ========================================
# 🔧 로컬 경로 설정
# ========================================
# 데이터 폴더 경로를 본인 환경에 맞게 수정하세요

DATA_DIR = r"data"  # 로컬 data 폴더 기준
OUTPUT_DIR = r"./kcelectra_output"  # 모델 저장 경로 (현재 폴더 기준)

print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 데이터 폴더: data
📁 출력 폴더: ./kcelectra_output


---
### 2️⃣ KcELECTRA 모델 학습

✔ Train으로 학습

✔ Test 셋은 학습에 쓰지 않음

✔ Test 셋은 Trainer의 evaluate()에 연결

✔ 자동으로 F1/Acc 등 출력

In [3]:
# 1) 환경 준비
import os
import random
import numpy as np
import torch

# 랜덤 시드 고정 → 매번 동일한 결과 재현 가능
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU가 감지되지 않았습니다. CPU로 실행됩니다.")

⚠️ GPU가 감지되지 않았습니다. CPU로 실행됩니다.


In [4]:
# 2) 데이터 불러오기 + 합치기
import pandas as pd

# 로컬 데이터 경로 설정 (datasets 폴더 기준 파일명)
abusive_path = os.path.join(DATA_DIR, "combined_abusive_shuffled_20k.csv")
nonabusive_path = os.path.join(DATA_DIR, "nonabusive_merged_shuffled_sample20000.csv")

# 파일 존재 확인
if not os.path.exists(abusive_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {abusive_path}")
if not os.path.exists(nonabusive_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {nonabusive_path}")

# 두 csv를 불러와 text, label만 선택
df_ab = pd.read_csv(abusive_path)[["text", "label"]]
df_non = pd.read_csv(nonabusive_path)[["text", "label"]]

# 데이터 합치기
df = pd.concat([df_ab, df_non], ignore_index=True)
df = df.dropna().reset_index(drop=True)

print(df.head())
print(f"\n📊 라벨 분포:\n{df['label'].value_counts()}")
print(f"\n총 데이터 수: {len(df)}개")

                                                text  label
0                                            ᄂᄀᄀᄈ 꺼져      1
1                                                씹퇭아      1
2                           누군지모르지만 요즘걸그룹창여들다발라버리네..      0
3  자살하면 다음생 좋은데 갈꺼같냐ㅋㅋ?절때 못가지금수저로는 더더욱 못태어남100프로흙...      1
4                                  니좆이*다 니좆이*다 니좆이*다      1

📊 라벨 분포:
label
0    22882
1    17118
Name: count, dtype: int64

총 데이터 수: 40000개


In [5]:
# 3) Train / Test 분리
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]    # 라벨 비율 유지
)

print(f"Train size: {len(train_df)}")
print(f"Test size : {len(test_df)}")

Train size: 32000
Test size : 8000


In [6]:
# 4) DatasetDict 구성 (train + validation=test)
from datasets import Dataset, DatasetDict

raw_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
})

print(raw_ds)

C:\ProgramData\anaconda3\envs\kcelec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 32000
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 8000
    })
})


In [7]:
# 5) 토크나이저 로드 + 토큰화
from transformers import AutoTokenizer

MODEL_ID = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# 토큰화 함수
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# DatasetDict에 토큰화 적용
tokenized = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")  # Trainer 규약에 맞게 labels로 변경
tokenized.set_format("torch")

print("✅ 토큰화 완료!")
print(tokenized)

Map: 100%|███████████████████████| 8000/8000 [00:01<00:00, 5130.66 examples/s]

✅ 토큰화 완료!
DatasetDict({
    train: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32000
    })
    validation: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
})


In [8]:
# 6) 클래스 가중치 계산 (데이터 불균형 보정)
import torch.nn as nn

labels_np = train_df["label"].values
pos = (labels_np == 1).sum()
neg = (labels_np == 0).sum()

# 단순 역수 기반 클래스 가중치
w0 = 1 / neg
w1 = 1 / pos
scale = 2 / (w0 + w1)
class_weights = torch.tensor([w0 * scale, w1 * scale], dtype=torch.float)

print(f"Class weights: {class_weights}")

Class weights: tensor([0.8559, 1.1441])


In [9]:
# 7) 모델 로드 + Trainer 정의 (evaluate 활성화)
# 실행하면 Some weights of ElectraForSequenceClassification were not initialized~~ 나오는데 무시해도 됩니다
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )

    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
        auc = roc_auc_score(labels, probs)
    except Exception:
        auc = float("nan")

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc,
    }

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2
)

# Weighted loss 적용되는 Trainer 정의
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Trainer 설정 (하이퍼파라미터는 여기에)
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",         # epoch마다 test셋으로 evaluate() 실행
    save_strategy="epoch",         # epoch마다 checkpoint 저장
    load_best_model_at_end=True,   # 가장 F1 좋은 모델 자동 로드
    metric_for_best_model="f1",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=base_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],  # test셋을 validation으로 사용 → evaluate 가능
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료!")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Trainer 설정 완료!


In [ ]:
# 8) 학습 수행 (train + evaluate 자동 출력)
trainer.train()

C:\ProgramData\anaconda3\envs\kcelec\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
# 9) 최종 테스트셋 성능 확인 (evaluate())
final_metrics = trainer.evaluate()
print("\n📊 Final Test Metrics:")
for key, value in final_metrics.items():
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

In [ ]:
# 10) 모델 저장
SAVE_DIR = os.path.join(OUTPUT_DIR, "best_model")

# 디렉토리 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# Trainer가 현재 가지고 있는 모델(=best model)을 저장
trainer.save_model(SAVE_DIR)

# 토크나이저도 함께 저장 (나중에 예측 시 필요)
tokenizer.save_pretrained(SAVE_DIR)

print("✅ 모델 저장 완료!")
print(f"📁 저장 경로: {SAVE_DIR}")

---
### 3️⃣ 테스트 문장 평가해보기

In [ ]:
# 테스트 문장 예측 함수
def predict(texts, threshold=0.5):
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    
    device = next(base_model.parameters()).device
    enc = {k: v.to(device) for k, v in enc.items()}
    
    with torch.no_grad():
        logits = base_model(**enc).logits
        probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        preds = (probs >= threshold).astype(int)
    return preds, probs

# 테스트 문장들
samples = [
    "야 이 개ㅅㄲ야",
    "야 이 개새야",
    "야 이 개새새야",
    "친절하게 도와주셔서 감사합니다.",
    "뭐라는거야 진짜 짜증나게",
    "아름다운 밤이네요.",
    "왜그런짓을하는거냥 씨댕",
    "ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅈㄹ 개욱기네",
    "아니 그게아니라니까요 ㄹㅇ로 구라 아니라고",
    "비꼬지마셈 발작자제좀",
    "ㅋㅋㅎㅇ",
    "아잇 ㅈ됐네",
    "아하! 컴마를 빼먹다니 정신나갔군",
    "님 정신나감? 18ㅋㅋ",
    "ㅇㅁ없는ㄴㅋㅋ 넌 나가라",
    "나 노래방 18번 업슴",
    "ㅈ1랄마쇼",
    "쌈@뽕하구만",
    "애미걸고 ㄹㅇ임",
    "ㅇㅁ걸고 ㄹㅇ임",
    "엄마걸고 ㄹㅇ임",
    "@",
    "개발자",
    "개새우튀김",
    "물방개새우티김",
    "님 정신나감?",
    "얘 정신나간듯",
    "대 상 혁",
    "^0^"
]

preds, probs = predict(samples)

print("\n📝 예측 결과:")
print("-" * 60)
for text, pred, prob in zip(samples, preds, probs):
    label = "🚨 악성" if pred == 1 else "✅ 정상"
    print(f"{label} ({prob:.2%}): {text}")